In [1]:
import importlib
cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

In [2]:
from cimgraph.databases import ConnectionParameters
from cimgraph.databases import RDFlibConnection
from cimgraph.models import FeederModel
from cimgraph.models import NodeBreakerModel
from cimgraph.models import BusBranchModel
from test_get_all_edges import test_get_all_edges
import cimgraph.utils as utils

In [3]:
cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

# RDFLib File Reader Connection
params = ConnectionParameters(filename="./test_models/ieee13.xml", cim_profile='cimhub_2023', iec61970_301=8)
rdf = RDFlibConnection(params)

feeder_mrid = "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"
feeder = cim.Feeder(mRID = feeder_mrid)
network = FeederModel(connection=rdf, container=feeder, distributed=False)
utils.get_all_data(network)
utils.write_xml(network, "./test_output/ieee13test.xml")
rdf.libgraph.serialize(destination='./test_output/test13rdf.xml', format='xml')

ConnectivityNode
Terminal
EnergyConsumer
Breaker
ACLineSegment
PowerTransformer
Recloser
PowerElectronicsConnection
LoadBreakSwitch
LinearShuntCompensator
EnergySource
Fuse
Feeder
TopologicalNode
OperationalLimitSet
TransformerTankEnd
TapChangerControl
PowerTransformerEnd
TopologicalIsland
ACLineSegmentPhase
BaseVoltage
PerLengthPhaseImpedance
Location
PhaseImpedanceData
TransformerTank
TransformerTankInfo
RatioTapChanger
TransformerEndInfo
ShortCircuitTest
NoLoadTest
TransformerMeshImpedance
TransformerCoreAdmittance
EnergyConsumerPhase
LoadResponseCharacteristic
PowerElectronicsConnectionPhase
BatteryUnit
PhotovoltaicUnit
CurrentLimit
VoltageLimit
OperationalLimitType
PositionPoint
CoordinateSystem
SwitchPhase
LinearShuntCompensatorPhase
Substation
SubGeographicalRegion
GeographicalRegion


<Graph identifier=N6a4c8d6991e94665ae770eba5c556fca (<class 'rdflib.graph.Graph'>)>

In [4]:
from cimgraph.utils.write_csv import write_csv

write_csv(network, './test_output')

In [6]:
## Round-trip test
cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

# RDFLib File Reader Connection
params = ConnectionParameters(filename="./test_output/ieee13test.xml", cim_profile='cimhub_2023', iec61970_301=8)
rdf = RDFlibConnection(params)

feeder_mrid = "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"
feeder = cim.Feeder(mRID = feeder_mrid)
network = FeederModel(connection=rdf, container=feeder, distributed=False)
test_get_all_edges(network, cim)
utils.get_all_data(network)
utils.write_xml(network, "./test_output/ieee13test2.xml")
rdf.libgraph.serialize(destination='./test_output/test13rdf_2.xml', format='xml')

ConnectivityNode
Terminal
EnergyConsumer
Breaker
ACLineSegment
PowerTransformer
Recloser
PowerElectronicsConnection
LoadBreakSwitch
LinearShuntCompensator
EnergySource
Fuse
Feeder
ACLineSegmentPhase
BaseVoltage
PerLengthPhaseImpedance
Location
PhaseImpedanceData
PowerTransformerEnd
TransformerTank
TransformerTankEnd
TransformerTankInfo
RatioTapChanger
TransformerEndInfo
ShortCircuitTest
NoLoadTest
TransformerMeshImpedance
TransformerCoreAdmittance
TapChangerControl
EnergyConsumerPhase
LoadResponseCharacteristic
TopologicalNode
OperationalLimitSet
CurrentLimit
VoltageLimit
OperationalLimitType
Substation
CoordinateSystem
PositionPoint
TopologicalIsland
PowerElectronicsConnectionPhase
BatteryUnit
PhotovoltaicUnit
SwitchPhase
LinearShuntCompensatorPhase
SubGeographicalRegion
GeographicalRegion


<Graph identifier=N6dcd4ef0576340d48a7af5fa5ef221a7 (<class 'rdflib.graph.Graph'>)>

In [7]:
## Distributed model test

network = FeederModel(connection=rdf, container=feeder, distributed=True)
for switch_area in network.distributed_areas:
    test_get_all_edges(switch_area, cim)
    for sec_area in switch_area.distributed_areas:
        test_get_all_edges(sec_area, cim)


In [8]:
cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

# RDFLib File Reader Connection
params = ConnectionParameters(filename="./test_models/IEEE118_CIM.xml", cim_profile='cimhub_2023', iec61970_301=8)
rdf = RDFlibConnection(params)

feeder_mrid = "1783D2A8-1204-4781-A0B4-7A73A2FA6038" #IEEE 118 Bus"
feeder = cim.ConnectivityNodeContainer(mRID = feeder_mrid)
network = BusBranchModel(connection=rdf, container=feeder, distributed=False)
test_get_all_edges(network, cim)

In [9]:
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

# RDFLib File Reader Connection
params = ConnectionParameters(filename="./test_models/maple10nodebreaker.xml", cim_profile=cim_profile, iec61970_301=7)
rdf = RDFlibConnection(params)

feeder_mrid = "_EE4C60AE-550D-4599-92F4-022DF3118B3C"
geo = cim.GeographicalRegion(mRID = feeder_mrid)
network = NodeBreakerModel(connection=rdf, container=geo, distributed=True) 

for sr_area in network.distributed_areas[cim.SubGeographicalRegion].values():
    print("subregion", sr_area.container.name)
    test_get_all_edges(sr_area, cim)
    for sub_area in sr_area.distributed_areas[cim.Substation].values():
        print("substation", sub_area.container.name)
        test_get_all_edges(sub_area, cim)
        for vl_area in sub_area.distributed_areas[cim.VoltageLevel].values():
            print("voltage level", vl_area.container.name)
            test_get_all_edges(vl_area, cim)
        for feeder in sub_area.distributed_areas[cim.Feeder].values():
            print("feeder", feeder.container.name, feeder.container.mRID)
            test_get_all_edges(feeder, cim)



subregion small
substation maple10bus_sub2
voltage level SUB2_4.16_B1
voltage level SUB2_115.0_B1
voltage level SUB2_230.0_B1
voltage level SUB2_34.5_B1
feeder feeder_8 _9E985101-27AD-4FE4-B36A-EBECC98CDFB1
feeder feeder_10 _9E985101-27AD-4FE4-B36A-EBECC98CDFB3
feeder feeder_11 _9E985101-27AD-4FE4-B36A-EBECC98CDFB4
feeder feeder_9 _9E985101-27AD-4FE4-B36A-EBECC98CDFB2
substation maple10bus_sub1
voltage level SUB1_230.0_B1
voltage level SUB1_34.5_B1
voltage level SUB1_12.47_B1
voltage level SUB1_115.0_B1
feeder feeder_3 _9E985101-27AD-4FE4-B36A-EBECC98CDFA3
feeder feeder_4 _9E985101-27AD-4FE4-B36A-EBECC98CDFA4
feeder feeder_1 _9E985101-27AD-4FE4-B36A-EBECC98CDFA1
feeder feeder_5 _9E985101-27AD-4FE4-B36A-EBECC98CDFA5
feeder feeder_6 _9E985101-27AD-4FE4-B36A-EBECC98CDFA6
feeder feeder_7 _9E985101-27AD-4FE4-B36A-EBECC98CDFA7
feeder feeder_2 _9E985101-27AD-4FE4-B36A-EBECC98CDFA2


In [10]:
network = NodeBreakerModel(connection=rdf, container=geo, distributed=False)

utils.get_all_data(network)

GeographicalRegion
SubGeographicalRegion
Substation
PowerTransformer
Feeder
VoltageLevel
PowerTransformerEnd
Location
Terminal
Breaker
ConnectivityNode
EnergyConsumer
PowerElectronicsConnection
BaseVoltage
EnergySource
SynchronousMachine
TransformerMeshImpedance
TransformerCoreAdmittance
PositionPoint
CoordinateSystem
OperationalLimitSet
TopologicalNode
LoadResponseCharacteristic
PhotovoltaicUnit
ACLineSegment
VoltageLimit
CurrentLimit
TopologicalIsland
OperationalLimitType
